In [20]:
import pandas as pd
import numpy as np
from functools import reduce
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams 
import helper_functions as hf

In [25]:
pd.options.mode.chained_assignment = None  # default='warn'
teams_list = teams.get_teams()

teams_per = dict()

for team in teams_list:
    team_id = team['id']
    teams_per[team_id] = 0

seasons_teams = []
seasons_players = []
first_season = 2020
last_season = 2021
first_season_id = 20000 + first_season

print("Getting NBA Seasons Information...")
for i in range(first_season,last_season):
    season_i_teams = leaguegamelog.LeagueGameLog(season = str(i)).get_data_frames()[0]
    season_i_players = leaguegamelog.LeagueGameLog(season = str(i), player_or_team_abbreviation = 'P').get_data_frames()[0]
    seasons_teams.append(season_i_teams)
    seasons_players.append(season_i_players)
    print("{}/{}".format(i,last_season))


dfs = []

print("Cleaning the data...")

season_games = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_teams)
season_games_plyrs = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_players)
season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

season_games_plyrs['GAME_ID'] = pd.to_numeric(season_games_plyrs['GAME_ID'])
season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
season_games_plyrs['GAME_DATE'] = pd.to_datetime(season_games_plyrs['GAME_DATE'])

print('size', len(season_games.index))

Getting NBA Seasons Information...
2020/2021
Cleaning the data...
size 2160


In [26]:
print("Initializing ELOs...")

elo_dic = dict()

for team in teams_list:
    elo_dic[team['id']] = 1500

matches_organized = []
matches_organized_lstm = []

n_last_games = 10
n_last_specific_games = 5

season_id = ''    
print('Getting historical odds...')
odds = hf.load_bets_csv()

Initializing ELOs...
Getting historical odds...


In [27]:
print("Creating CSV file of all games...")
for i, g in season_games.groupby(season_games.index // 2):
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break

    if season_id != '' and season_id != g.iloc[[0],:].iloc[0]['SEASON_ID']:
        hf.reset_season_elo(season_id, g, elo_dic)

    season_id = g.iloc[[0],:].iloc[0]['SEASON_ID']
        
    game_id = g.iloc[[0],:].iloc[0]['GAME_ID']
    game_date = g.iloc[[0],:].iloc[0]['GAME_DATE']

    team_a_id = g.iloc[[0],:].iloc[0]['TEAM_ID']
    team_b_id = g.iloc[1:2,:].iloc[0]['TEAM_ID']

    team_a_abbv = g.iloc[[0],:].iloc[0]['TEAM_ABBREVIATION']
    team_b_abbv = g.iloc[1:2,:].iloc[0]['TEAM_ABBREVIATION']
    
    winner = 'A' if g.iloc[[0],:].iloc[0]['WL'] == 'W' else 'B'
    
    # Update ELO after stats computed
    team_a_pts = g.iloc[[0],:].iloc[0]['PTS']
    team_b_pts = g.iloc[1:2,:].iloc[0]['PTS']
    elo_a = elo_dic[team_a_id]
    elo_b = elo_dic[team_b_id]

    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        team_b_odds, team_a_odds = hf.get_teams_odds(team_b_id, team_a_id, game_date, odds)
    else:
        team_a_odds, team_b_odds = hf.get_teams_odds(team_a_id, team_b_id, game_date, odds)

    team_a_previous_games = season_games.loc[(season_games['TEAM_ID'] == team_a_id) & (season_games['GAME_DATE'] < game_date)]
    team_b_previous_games = season_games.loc[(season_games['TEAM_ID'] == team_b_id) & (season_games['GAME_DATE'] < game_date)]
    team_a_season_games = team_a_previous_games.loc[team_a_previous_games['SEASON_ID'] == season_id]
    team_b_season_games = team_b_previous_games.loc[team_b_previous_games['SEASON_ID'] == season_id]

    # Getting teams last 10 games
    team_a_previous_n_games = team_a_season_games.iloc[-n_last_games:,:]
    team_b_previous_n_games = team_b_season_games.iloc[-n_last_games:,:]

    if len(team_a_previous_games.index) > 0:
        if team_a_previous_games.iloc[-1]['GAME_ID'] == g.iloc[[0],:].iloc[0]['GAME_ID']:
            break

    if not (int(season_id) >= first_season_id and len(team_a_previous_n_games.index) >= 5 and len(team_b_previous_n_games.index) >= 5 and team_a_odds != None and team_b_odds != None):
        print("Not enough games.")
        hf.update_elo(winner, elo_a, elo_b, elo_dic, team_a_id, team_b_id, team_a_pts, team_b_pts)
        continue

    # Getting player information
    teams_per[team_a_id] = hf.get_team_per_mean(team_a_id, game_id, game_date, season_id, season_games_plyrs)
    teams_per[team_b_id] = hf.get_team_per_mean(team_b_id, game_id, game_date, season_id, season_games_plyrs)

    # Season Win Percentage
    team_a_season_pct = hf.get_wl_pct(team_a_season_games)[0]
    team_b_season_pct = hf.get_wl_pct(team_b_season_games)[0]

    # Poins Conceded
    team_a_previous_games_pts_conceded = hf.team_points_conceded(team_a_previous_n_games, season_games)
    team_b_previous_games_pts_conceded = hf.team_points_conceded(team_b_previous_n_games, season_games)

    stats_team_a = hf.get_team_stats (team_a_previous_n_games, team_a_previous_games_pts_conceded, team_a_season_pct, elo_a, teams_per[team_a_id], team_a_odds)
    stats_team_b = hf.get_team_stats (team_b_previous_n_games, team_b_previous_games_pts_conceded, team_b_season_pct, elo_b, teams_per[team_b_id], team_b_odds)

    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        matches_organized.append([season_id, game_date, team_b_abbv, team_a_abbv] + stats_team_b + stats_team_a + [1 if winner == 'B' else 0])
    else:
        matches_organized.append([season_id, game_date, team_a_abbv, team_b_abbv] + stats_team_a + stats_team_b + [1 if winner == 'A' else 0])

    matches_organized_lstm.append([team_a_abbv, team_a_id, game_date, team_a_pts, team_b_pts, g.iloc[[0],:].iloc[0]['FG_PCT'], g.iloc[[0],:].iloc[0]['FG3_PCT'], 
                    g.iloc[[0],:].iloc[0]['FT_PCT'], g.iloc[[0],:].iloc[0]['REB'], g.iloc[[0],:].iloc[0]['TOV'],
                    team_a_season_pct, elo_a, elo_b,
                     teams_per[team_a_id], team_a_odds, team_b_odds, 1 if winner == 'A' else 0])

    matches_organized_lstm.append([team_b_abbv, team_b_id, game_date, team_b_pts, team_a_pts, g.iloc[1:2,:].iloc[0]['FG_PCT'], g.iloc[1:2,:].iloc[0]['FG3_PCT'], 
                    g.iloc[1:2,:].iloc[0]['FT_PCT'], g.iloc[1:2,:].iloc[0]['REB'], g.iloc[1:2,:].iloc[0]['TOV'],
                    team_b_season_pct, elo_b, elo_a,
                     teams_per[team_b_id], team_b_odds, team_a_odds, 1 if winner == 'B' else 0])


    hf.update_elo(winner, elo_a, elo_b, elo_dic, team_a_id, team_b_id, team_a_pts, team_b_pts)

Creating CSV file of all games...
0/1080
Not enough games.
1/1080
Not enough games.
2/1080
Not enough games.
3/1080
Not enough games.
4/1080
Not enough games.
5/1080
Not enough games.
6/1080
Not enough games.
7/1080
Not enough games.
8/1080
Not enough games.
9/1080
Not enough games.
10/1080
Not enough games.
11/1080
Not enough games.
12/1080
Not enough games.
13/1080
Not enough games.
14/1080
Not enough games.
15/1080
Not enough games.
16/1080
Not enough games.
17/1080
Not enough games.
18/1080
Not enough games.
19/1080
Not enough games.
20/1080
Not enough games.
21/1080
Not enough games.
22/1080
Not enough games.
23/1080
Not enough games.
24/1080
Not enough games.
25/1080
Not enough games.
26/1080
Not enough games.
27/1080
Not enough games.
28/1080
Not enough games.
29/1080
Not enough games.
30/1080
Not enough games.
31/1080
Not enough games.
32/1080
Not enough games.
33/1080
Not enough games.
34/1080
Not enough games.
35/1080
Not enough games.
36/1080
Not enough games.
37/1080
Not en

759/1080
760/1080
761/1080
762/1080
763/1080
764/1080
765/1080
766/1080
767/1080
768/1080
769/1080
770/1080
771/1080
772/1080
773/1080
774/1080
775/1080
776/1080
777/1080
778/1080
779/1080
780/1080
781/1080
782/1080
783/1080
784/1080
785/1080
786/1080
787/1080
788/1080
789/1080
790/1080
791/1080
792/1080
793/1080
794/1080
795/1080
796/1080
797/1080
798/1080
799/1080
800/1080
801/1080
802/1080
803/1080
804/1080
805/1080
806/1080
807/1080
808/1080
809/1080
810/1080
811/1080
812/1080
813/1080
814/1080
815/1080
816/1080
817/1080
818/1080
819/1080
820/1080
821/1080
822/1080
823/1080
824/1080
825/1080
826/1080
827/1080
828/1080
829/1080
830/1080
831/1080
832/1080
833/1080
834/1080
835/1080
836/1080
837/1080
838/1080
839/1080
840/1080
841/1080
842/1080
843/1080
844/1080
845/1080
846/1080
847/1080
848/1080
849/1080
850/1080
851/1080
852/1080
853/1080
854/1080
855/1080
856/1080
857/1080
858/1080
859/1080
860/1080
861/1080
862/1080
863/1080
864/1080
865/1080
866/1080
867/1080
868/1080
869/1080
8

In [28]:
final_df = pd.DataFrame(matches_organized, columns=['SEASON_ID', 'GAME_DATE', 'TEAM_A', 'TEAM_B',
                                                    'PTS_A', 'PTS_CON_A', 'FG_PCT_A', 'FG3_PCT_A', 'FT_PCT_A', 'REB_A', 'TOV_A', 'SEASON_A_PCT', 'ELO_A', 'PER_A', 'ODDS_A',
                                                    'PTS_B', 'PTS_CON_B', 'FG_PCT_B', 'FG3_PCT_B', 'FT_PCT_B', 'REB_B', 'TOV_B', 'SEASON_B_PCT', 'ELO_B', 'PER_B', 'ODDS_B',
                                                    'WINNER'])
final_df_lstm = pd.DataFrame(matches_organized_lstm, columns=['TEAM_ABBV', 'TEAM_ID', 'DATE',
                                                    'PTS_A', 'PTS_CON_A', 'FG_PCT_A', 'FG3_PCT_A', 'FT_PCT_A', 'REB_A', 'TOV_A', 
                                                    'SEASON_A_PCT', 'ELO_A', 'ELO_OPP', 'PER_A', 'ODDS_A', 'ODDS_OPP',
                                                    'WINNER'])
final_df.to_csv('../data/{}-{}.csv'.format(first_season, last_season-1))
final_df_lstm.to_csv('../data/LSTM/{}-{}.csv'.format(first_season, last_season-1))